In [ ]:
#Install if necessary
#!pip install kmodes
#!pip install statsmodels


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/103.0 kB ? eta -:--:--
   --- ------------------------------------ 10.2/103.0 kB ? eta -:--:--
   ----------- --------------------------- 30.7/103.0 kB 366.2 kB/s eta 0:00:01
   --------------- ----------------------- 41.0/103.0 kB 389.9 kB/s eta 0:00:01
   ---------------------------------- ---- 92.2/103.0 kB 588.6 kB/s eta 0:00:01
   -------------------------------------- 103.0/103.0 kB 560.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/224.5 kB ? eta -:--:--
   ---------- ----------------------------- 61.4/224.5 kB 2.9 MB/s eta 0:00:01
   ------------------ --------------------- 102.4/224.5 kB 1.1 MB/s eta 0:00:01
   -------------------------------- ------- 184.3/224.5 kB 1.7 MB/s eta 0:00:01
   -------------------------------------- - 215.0/224.5 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 224.5/224.5 kB 1.1 MB/s e


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\lewis\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
#Import libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


In [2]:
# Set pandas display options to show all columns
pd.set_option('display.max_columns', None)


# Load the dataset
df = pd.read_csv('preprocessed_hotel_booking.csv')
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,arrival_date,days_before_arrival
0,Resort Hotel,0,342,2015,7,27,1,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,no agent,0,Transient,0.0,0,0,Check-Out,2015-07-01,2015-07-01,0
1,Resort Hotel,0,737,2015,7,27,1,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,no agent,0,Transient,0.0,0,0,Check-Out,2015-07-01,2015-07-01,0
2,Resort Hotel,0,7,2015,7,27,1,0,1,1,0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,no agent,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,-1
3,Resort Hotel,0,13,2015,7,27,1,0,1,1,0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,has agent,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,-1
4,Resort Hotel,0,14,2015,7,27,1,0,2,2,0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,has agent,0,Transient,98.0,0,1,Check-Out,2015-07-03,2015-07-01,-2


In [3]:
# Convert date columns to datetime
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])
df['arrival_date'] = pd.to_datetime(df['arrival_date'])

### **Cancellation Prediction Model**

In [4]:
df1 = df
df1 = pd.get_dummies(df1, columns=['arrival_date_month', 'meal', 'market_segment', 'distribution_channel', 'agent', 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type'])

In [5]:
# Label encode the country column
le = LabelEncoder()
df1['country'] = le.fit_transform(df1['country'])
df1['hotel'] = le.fit_transform(df1['hotel'])

In [6]:
df1 = df1.select_dtypes(exclude=['category'])
df1

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,country,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,arrival_date,days_before_arrival,arrival_date_month_1,arrival_date_month_2,arrival_date_month_3,arrival_date_month_4,arrival_date_month_5,arrival_date_month_6,arrival_date_month_7,arrival_date_month_8,arrival_date_month_9,arrival_date_month_10,arrival_date_month_11,arrival_date_month_12,meal_BB,meal_FB,meal_HB,meal_SC,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,agent_has agent,agent_no agent,reserved_room_type_A,reserved_room_type_B,reserved_room_type_C,reserved_room_type_D,reserved_room_type_E,reserved_room_type_F,reserved_room_type_G,reserved_room_type_H,reserved_room_type_L,assigned_room_type_A,assigned_room_type_B,assigned_room_type_C,assigned_room_type_D,assigned_room_type_E,assigned_room_type_F,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,assigned_room_type_L,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
0,1,0,342,2015,27,1,0,0,2,0,0,135,0,0,0,3,0,0.00,0,0,Check-Out,2015-07-01,2015-07-01,0,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False
1,1,0,737,2015,27,1,0,0,2,0,0,135,0,0,0,4,0,0.00,0,0,Check-Out,2015-07-01,2015-07-01,0,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False
2,1,0,7,2015,27,1,0,1,1,0,0,59,0,0,0,0,0,75.00,0,0,Check-Out,2015-07-02,2015-07-01,-1,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False
3,1,0,13,2015,27,1,0,1,1,0,0,59,0,0,0,0,0,75.00,0,0,Check-Out,2015-07-02,2015-07-01,-1,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False
4,1,0,14,2015,27,1,0,2,2,0,0,59,0,0,0,0,0,98.00,0,1,Check-Out,2015-07-03,2015-07-01,-2,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,True,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [7]:
X = df1.drop(columns=['is_canceled','reservation_status','country', 
                     'reservation_status_date', 'arrival_date'])
y = df1['is_canceled']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### **Random Forest Classifier**

In [10]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=42)

In [12]:
with open('rf_clf.pkl', 'wb') as f:
    pickle.dump(rf_clf, f)